# Cortex Chat + TruLens

This quickstart assumes you already have a Cortex Search Service started and JWT token created.

## Set JWT Token

In [ ]:
import os
os.environ["SNOWFLAKE_JWT"] = "..."

In [ ]:
import requests
import os
import json

def handle_cortex_chat_response(response):
    for line in response.iter_lines():
        if line:
            decoded_line = line.decode('utf-8')
            if decoded_line.startswith('data:'):
                try:
                    data = json.loads(decoded_line[5:])
                    if data['delta']['content'][0]['type'] == "text":
                        print(data['delta']['content'][0]['text']['value'], end='', flush=True)
                    if data['delta']['content'][0]['type'] == "citation":
                        print(f"\n", json.dumps(data['delta']['content'][0]['citation'], indent=4))
                    if data['delta']['content'][0]['type'] == "debug_info":
                        print(f"\n",json.dumps(data['delta']['content'][0]['debug_info'], indent=4))
                except json.JSONDecodeError:
                    print(f"Error decoding JSON: {decoded_line}")
                    
def cortex_chat(query):
    url = "https://cortexsearchexternal.qa6.us-west-2.aws.snowflakecomputing.com/api/v2/cortex/chat"
    headers = {
        'X-Snowflake-Authorization-Token-Type': 'KEYPAIR_JWT',
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f"Bearer {os.environ.get('SNOWFLAKE_JWT')}"
    }
    data = {
        "query": query,
        "model": "mistral-7b",
        "debug": True,
        "search_services": [{
            "name": "llmops_db.llmops_schema.llmops_cortex_search_service",
            "max_results": 100
        }],
        "prompt": "Answer the question: {{.Question}}",
    }

    response = requests.post(url, headers=headers, json=data, stream=True)
    if response.status_code == 200:
        handle_cortex_chat_response(response)
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [ ]:
# Example usage
user_query = "How do I create an app deployed to streamlit community cloud?"
cortex_chat(user_query)